In [410]:
colours = ['red', 'yellow', 'green', 'blue']

In [411]:
class Card:
    def __init__(self, colour, value):
        self.colour = colour
        self.value = value
        
    def __repr__(self):
        return self.colour + ' ' + str(self.value)
    
    def is_playable(self, prev_card):
        if(prev_card.colour == self.colour or prev_card.value == self.value): return True
        
        return False
        
class PowerCard(Card):
    def __init__(self, card_id, colour='', display_name='', special_attr=None, first_playable=True, custom_is_playable=None, play_handler=None):
        super().__init__(colour, card_id)
        self.card_id = card_id
        self.special_attr = special_attr
        self.first_playable = first_playable
        self.custom_is_playable = custom_is_playable

        if display_name == '':
            display_name = colour + ' ' + card_id

        self.display_name = display_name
        self.play_handler = play_handler
        
    def __repr__(self):
        return self.display_name
        

    def is_playable(self, prev_card):
        if self.custom_is_playable is not None:
            return self.custom_is_playable(prev_card)

        if self.colour != "" and self.colour == prev_card.colour:
            return True

        return False
    
    def set_colour(self, colour):
        self.colour = colour
        self.display_name = colour + ' ' + self.card_id
        
    def handle_played(self, game_instance):
        if self.play_handler is not None: 
            self.play_handler(game_instance, self)
            return
        
        pass

        

In [412]:
def handle_draw_card(game_instance, card):
    next_player = game_instance.get_next_player()
    pickup_amount = card.special_attr
    
    game_instance.handle_pickup(next_player, pickup_amount, 'Pickup ' + str(pickup_amount))

def create_draw_card(draw_amount, colour):
    display_name = colour + ' draw ' + str(draw_amount)
    return PowerCard('draw', colour, display_name, draw_amount, play_handler=handle_draw_card)

In [413]:
def handle_skip_card(game_instance, card):
    next_player = game_instance.get_next_player()
    game_instance.skip_player(next_player)

def create_skip_card(colour):
    return PowerCard('skip', colour, play_handler=handle_skip_card)

In [414]:
def handle_reverse_card(game_instance, card):
    game_instance.flip_play_direction()

def create_reverse_card(colour):
    return PowerCard('reverse', colour, play_handler=handle_reverse_card)

In [415]:
def wild_card_playable(prev_card):
    return True

def handle_wild_card(game_instance, card):
    colour = None
    
    while colour == None:
        input_colour = str(input("Enter a valid colour for the wild card (" + str(colours) + "): ")).lower()
        
        if input_colour in colours: colour = input_colour
    
    card.set_colour(colour)

def create_wild_card():
    return PowerCard('wild', first_playable=False, custom_is_playable=wild_card_playable, play_handler=handle_wild_card)

In [416]:
def handle_special_wild_card(game_instance, card):
    handle_wild_card(game_instance, card)
    handle_draw_card(game_instance, card)

def create_special_wild_card(draw_amount):
    display_name = 'wild draw ' + str(draw_amount)
    return PowerCard('special_wild', special_attr=draw_amount, display_name=display_name, first_playable=False, custom_is_playable=wild_card_playable, play_handler=handle_special_wild_card)

In [417]:
def create_colour_card_set(colour, number_card_ranges, colour_item_count):
    lower = number_card_ranges[0]
    upper = number_card_ranges[1]
    
    if(lower >= upper): return []
    
    card_set = []
    
    for i in range(lower, upper):
        for j in range(colour_item_count):
            card = Card(colour, i)
            card_set.append(card)
            
    for i in range(colour_item_count):
        card_set.append(create_draw_card(2, colour))
        card_set.append(create_reverse_card(colour))
        card_set.append(create_skip_card(colour))
    
    return card_set

In [418]:
def create_deck(colours, wild_card_count = 4):
    deck = []
    
    for colour in colours:
        colour_set = create_colour_card_set(colour, [0,9], 2)
        deck.extend(colour_set)
    
    for i in range(wild_card_count):
        deck.append(create_wild_card())
        deck.append(create_special_wild_card(4))
        
    return deck

In [419]:
import random

In [420]:
def shuffle_deck(deck, played_cards = []):
    shuffled_deck = deck.copy()
    random.shuffle(shuffled_deck)
    
    while True:
        revealed_card = shuffled_deck.pop(0)
        played_cards.append(revealed_card)
        
        revealed_card_playable = True
        
        if isinstance(revealed_card, PowerCard) and not revealed_card.first_playable:
            revealed_card_playable = False
            
        if revealed_card_playable: break
    
    return shuffled_deck, played_cards

In [421]:
def deal_cards(deck, player_count, cards_per_player):
    total_cards_needed = player_count * cards_per_player

    if total_cards_needed > len(deck):
        raise ValueError("Not enough cards to distribute evenly among players.")

    player_hands = [deck.pop(0) for _ in range(total_cards_needed)]

    return [player_hands[i:i + cards_per_player] for i in range(0, total_cards_needed, cards_per_player)]
        

In [422]:
def handle_deck_exhausted(deck, played_cards):
    print("The game deck has been exhausted! Now shuffling played cards...")
    
    new_deck = deck + played_cards
    
    return shuffle_deck(new_deck, [])

In [427]:
class UNOGame:
    def __init__(self, player_count, cards_per_player):
        self.player_count = player_count
        self.current_deck, self.played_cards = shuffle_deck(create_deck(['yellow', 'green', 'blue', 'red']))
        self.player_hands = deal_cards(self.current_deck, player_count, cards_per_player)
        self.current_player = 0
        self.game_active = False
        self.play_direction = 1
        self.skipped_players = []
        
    def get_next_player(self):
        if self.play_direction == 1:
            next_player = self.current_player + 1
            if next_player > self.player_count:
                next_player = 1
        elif self.play_direction == -1:
            next_player = self.current_player - 1
            if next_player < 1:
                next_player = self.player_count
        else:
            raise ValueError("Invalid play_direction. It should be either 1 or -1.")

        return next_player
    
    def flip_play_direction(self):
        if self.play_direction == 1:
            self.play_direction = -1
            return
        
        self.play_direction = 1
    
    def skip_player(self, player):
        self.skipped_players.append(player)
        
    def play_hand(self, player):
        last_played_card = self.get_last_played_card()
        player_hand = self.player_hands[player-1]
        playable_cards = self.get_playable_cards(player)
        
        print('\nPlayer ' + str(player) + ", it is your turn to play!")
        print('The last played card was ' + str(last_played_card))
        
        print("Here are your all cards. (Not all are playable): " + str(player_hand))
        card_idx = int(input("These are your playable cards. Please enter the number for the card to play: " + str(playable_cards))) -1
    
        card = playable_cards[card_idx]
        
        self.played_cards.append(card)
        player_hand.remove(card)
        
        print("You played " + str(card) + ', leaving you with ' + str(len(player_hand)) + " card(s) remaining!")
        if isinstance(card, PowerCard):
            card.handle_played(self)
            
        if self.check_win_condition(player):
            self.handle_win(player)
                   
    def handle_pickup(self, player, amount=1, reason='No playable cards'):
        if(amount >= len(self.current_deck)):
            self.current_deck, self.played_cards = handle_deck_exhausted(self.current_deck, self.played_cards)
        
        picked_up_cards = self.current_deck[:amount]
        del self.current_deck[:amount]
        
        self.player_hands[player-1] += picked_up_cards
        
        print('\nPlayer ' + str(player) + ' picked up ' + str(amount) + ' card(s) for the reason: ' + reason)
        print(str(picked_up_cards))
        
        
    def get_last_played_card(self):
        num_cards = len(self.played_cards)
        
        if num_cards == 0: return None
        return self.played_cards[num_cards-1]
        
    def get_playable_cards(self, player):
        player_cards = self.player_hands[player-1]
        playable_cards = [card for card in player_cards if card.is_playable(self.get_last_played_card())]
        
        return playable_cards
        
    def handle_round(self):       
        for player_itr in range(1, self.player_count + 1):
            if not self.game_active: break
            
            player = self.get_next_player()
            self.current_player = player
            
            if player in self.skipped_players:
                print("Player " + str(player) + ' was skipped!')
                self.skipped_players.remove(player)
                continue
            
            playable_cards = self.get_playable_cards(player)
                   
            if len(playable_cards) == 0:
                self.handle_pickup(player)
            else: self.play_hand(player)
        self.current_player = 0
        
    def check_win_condition(player):
        player_hand = self.player_hands[player-1]
        finished = len(player_hand) == 0
        
        return finished
    
    def handle_win(player):
        print("Player " + player + " has won!")
        game_active = False
        
    def start_game(self):
        self.current_player = 0
        self.game_active = True
                   
        while self.game_active:
            self.handle_round()
                   
        

In [ ]:
uno_game = UNOGame(4, 7)
uno_game.start_game()


Player 1, it is your turn to play!
The last played card was red 8
Here are your all cards. (Not all are playable): [yellow reverse, yellow 7, green reverse, yellow skip, red 6, blue 5, yellow 2]
